# Implementing Slowly Changing Dimension (SCD) Type 1 in pandas dataframes

__Task:__ Implement Slowly Changing Dimension (SCD) Type 1 in Pandas. Update the existing records with new information when applicable. 
If no changes are present, keep the existing data.

1. Determining the updated records involved
2. Setting the pk column as index and ensuring no new variable is returned.
3. update existing dataframe with new changes and add new columns by identifying what new data is in existing using boolean filter flipped i.e ~new_data + isin(existing_data) = false
4. Reset index to get customer_id back as a column, then sort values by customer_id

Data Frames Description:

* Existing Data: Contains customer information such as customer_id, name, address, and last_updated.

* New Data: Contains updates to the customer information, with changes in name, address, or new customers.


In [4]:

#determine updated records (changed name or address)
key = 'customer_id'
columns_to_compare = [col for col in existing_data.columns
                      if col not in [key, 'last_updated']]

#merge on customer_id to compare
merged = pd.merge(existing_data, new_data, on= 'customer_id', how='outer', suffixes=('_old','_new'))
merged

,name_old,address_old,last_updated_old,name_new,address_new,last_updated_new
customer_id,,,,,,
1,John Doe,123 Elm St,2023-04-01,John Doe,123 Elm St,2023-04-01
2,Jane Doe,456 Oak St,2023-04-01,Jane Doe,456 Oak St,2023-04-01
3,Jim Brown,789 Pine St,2023-03-01,NaN,NaN,NaN
4,NaN,NaN,NaN,Lucy Green,101 Maple St,2023-04-01


In [6]:

import pandas as pd

#determine updated records (changed name or address)
key = 'customer_id'
columns_to_compare = [col for col in existing_data.columns
                      if col not in [key, 'last_updated']]

#merge on customer_id to compare
merged = pd.merge(existing_data, new_data, on= 'customer_id', how='outer', suffixes=('_old','_new'))

existing_data = pd.DataFrame([
 (1, "John Doe", "123 Elm St", "2023-01-01"),
 (2, "Jane Smith", "456 Oak St", "2023-02-01"),
 (3, "Jim Brown", "789 Pine St", "2023-03-01")
], columns=["customer_id","name","address", "last_updated"])


new_data = pd.DataFrame([
 (1, "John Doe", "123 Elm St", "2023-04-01"), #no change
 (2, "Jane Doe", "456 Oak St", "2023-04-01"), #new change
 (4, "Lucy Green", "101 Maple St", "2023-04-01")
], columns=["customer_id","name","address", "last_updated"])

existing_data.set_index("customer_id", inplace=True)
new_data.set_index("customer_id", inplace=True)

# Overwrite (SCD Type 1): update existing and add new
existing_data.update(new_data)  # Updates matching rows with new data in the existing data
final_df = pd.concat([existing_data, new_data[~new_data.index.isin(existing_data.index)]])

# Reset index to get customer_id back as a column
final_df = final_df.reset_index().sort_values("customer_id")

final_df.head()

,customer_id,name,address,last_updated
0,1,John Doe,123 Elm St,2023-04-01
1,2,Jane Doe,456 Oak St,2023-04-01
2,3,Jim Brown,789 Pine St,2023-03-01
3,4,Lucy Green,101 Maple St,2023-04-01


End